In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import itertools
import matplotlib.pyplot as plt
import glob
from PIL import Image
import shutil
import os
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
transform = {'train': transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(200),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(200),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'eval': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(200),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

In [ ]:
class PlantSeedlingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.x = []
        self.y = []
        self.transform = transform
        self.num_classes = 0
        if self.root_dir.name=='train':
            for i, _dir in enumerate(self.root_dir.glob('*')):
                for file in _dir.glob('*'):
                    self.x.append(file)
                    self.y.append(i)
                self.num_classes+=1
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = Image.open(self.x[index]).convert('RGB')
        if self.transform: image = self.transform(image)
        return image, self.y[index]

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 12)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.)

In [ ]:
train_set = PlantSeedlingDataset(Path('../input/').joinpath('train'), transforms['train'])
eval_set = PlantSeedlingDataset(Path('../input/').joinpath('train'), transforms['train'])
train_set, val_set = torch.utils.data.dataset.random_split(train_set, [int((len(train_set)*4)/5), int(len(train_set)/5)])
train_dl = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
val_dl = DataLoader(val_set, batch_size=16, shuffle=False, num_workers=0)
eval_dl = DataLoader(eval_set, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
def score(model, dataloader):
    calc_loss = 0.
    calc_correct = 0.
    calc_count = 0.

    for data in dataloader:
        input, label = data
        input = Variable(input.cuda())
        label = Variable(label.cuda())
        output = model(input)
        _, pred = torch.max(output.data, 1)
        loss = criterion(output, label)
        calc_loss +=loss.item()
        calc_correct +=torch.sum(pred==label.data)
        calc_count+=output.data.shape[0]
    return round(calc_loss/calc_count, 4)

In [ ]:
def train(train_loader, val_loader, eval_loader, epochs):
    for epoch in range(epochs):
        calc_loss = 0.
        calc_correct = 0.
        calc_count = 0.
        for data in train_loader:
            inputs, labels = data
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            calc_loss+=loss.item()
            calc_correct+=torch.sum(preds==labels.data)
            calc_count+=outputs.data.shape[0]
        print('-------------')
        print('epoch:', epoch)
        print('run loss:', round(calc_loss/calc_count, 4))
        print('train loss:', score(model, eval_loader))
        print('val loss:', score(model, val_loader))
    return model

In [ ]:
def adjust_lr(optimizer, value):
    for param_group in optimizer.param_groups:
        param_group['lr'] = value
    return optimizer

In [ ]:
model = train(train_dl, val_dl, eval_dl, 5)

In [ ]:
optimizer = adjust_lr(optimizer, 0.00005)
model = train(train_dl, val_dl, eval_dl, 5)

In [ ]:
import pandas as pd

In [ ]:
!ls ../

In [ ]:
def test():
    data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    test_root = Path('../input/')
    classes = [_dir.name for _dir in test_root.joinpath('train').glob('*')]
    sample_submission = pd.read_csv(str(test_root.joinpath('sample_submission.csv')))
    submission = sample_submission.copy()
    for i, filename in enumerate(sample_submission['file']):
        image = Image.open(str(test_root.joinpath('test').joinpath(filename))).convert('RGB')
        image = data_transform(image).unsqueeze(0)
        inputs = Variable(image.cuda())
        output = model(inputs)
        _, preds = torch.max(output.data, 1)
        submission['species'][i] = classes[preds[0]]
    submission.to_csv('submission.csv', index=False)

In [ ]:
test()

In [ ]:
df = pd.read_csv('submission.csv')
df

# References:
1. https://pytorch.org/tutorials/
2. https://github.com/magical2world/Plant-seedlings-recognition-through-fine-tuning-resnet18-using-pytorch
3. https://github.com/remorsecs/Kaggle-Plant-Seedlings-Classification-Example
4. https://github.com/brightertiger/kaggle/tree/master/playground/plant-seed